In [1]:
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.5. Cluster Distribution Analyses/1.5.1. Wilcoxon Rank Sum/Input')
Output = ('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.5. Cluster Distribution Analyses/1.5.1. Wilcoxon Rank Sum/Output')
cur_date = "120621"

library(readxl)
library(data.table)
library(ggplot2)
library(factoextra)
library(janitor)
library(dplyr)
library(tidyverse)
library(gridExtra)
library(cluster)
library(vegan)
library(fpc)
library(ggdendro)

#reading in file
cytokines <- data.frame(read_excel("CytokineData_102920.xlsx", sheet = 2))

#cluster assignments for each compartment 
NELF_clus <- data.frame(read_excel("021421NELF_cluster_assignments.xlsx")) 
NLF_clus <- data.frame(read_excel("021421NLF_cluster_assignments.xlsx"))
Serum_clus <- data.frame(read_excel("021421Serum_cluster_assignments.xlsx"))
Sputum_clus <- data.frame(read_excel("021421Sputum_cluster_assignments.xlsx"))

#consensus cluster assignments 
consensus_clus <- data.frame(read_excel("110821consensus_clusters.xlsx")) 

Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test



Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.0.6     ✔ purrr   0.3.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr:

Assigning the same baseline consensus clusters to e-cig and cigarette smokers and running wilcoxon rank sum tests to see if they differ. 

# Eigencytokines

In [2]:
get_eigencytokines = function(smoking_status){
    # Separating the cytokine data into consensus cluster dfs
    cytokines <- cytokines %>% 
        #removing serum data since that wasn't used to derive the consensus clusters
        #also removing cytokines that weren't in any of the consensus clusters
        filter(Group == smoking_status, Protein %in% drop_na(consensus_clus)$Cytokine, Compartment != 'Serum') %>% 
        group_by(Compartment) %>% 
        group_split
    NELF <- cytokines[[1]]
    NLF <- cytokines[[2]]
    Sputum <- cytokines[[3]]

    # reshaping data 
    NELF <- reshape2::dcast(NELF, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 
    NLF <- reshape2::dcast(NLF, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 
    Sputum <- reshape2::dcast(Sputum, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 

    # the scale function operates across columns 
    NELF_scaled <- NELF %>% 
      scale() %>% 
      as.data.frame()
    NLF_scaled <- NLF %>% 
      scale() %>% 
      as.data.frame()
    Sputum_scaled <- Sputum %>% 
      scale() %>% 
      as.data.frame()

    #transpose cytokine data for each compartment
    NELF <- as.data.frame(t(NELF))
    NLF <- as.data.frame(t(NLF)) 
    Sputum <- as.data.frame(t(Sputum)) 

    #renaming first column, grouping and splitting by "Cluster" column
    NELF_clus <- NELF_clus %>% 
      group_by(Cluster) %>% 
      group_split
    NLF_clus <- NLF_clus %>% 
      group_by(Cluster) %>% 
      group_split
    Sputum_clus <- Sputum_clus %>% 
      group_by(Cluster) %>% 
      group_split

    #making dfs for each cluster for PCA analysis 
    NELF_1 <- NELF_clus[[1]]
    NELF_2 <- NELF_clus[[2]]
    NELF_3 <- NELF_clus[[3]]

    NLF_1 <- NLF_clus[[1]]
    NLF_2 <- NLF_clus[[2]]
    NLF_3 <- NLF_clus[[3]]

    Sputum_1 <- Sputum_clus[[1]]
    Sputum_2 <- Sputum_clus[[2]]
    Sputum_3 <- Sputum_clus[[3]]

    #making df with subjects' cytokine concentration data for each cluster 
    NELF_1 <- NELF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NELF_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NELF_2 <- NELF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NELF_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NELF_3 <- NELF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NELF_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    NLF_1 <- NLF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NLF_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NLF_2 <- NLF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NLF_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NLF_3 <- NLF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NLF_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    Sputum_1 <- Sputum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Sputum_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Sputum_2 <- Sputum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Sputum_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Sputum_3 <- Sputum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Sputum_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    #PCA on each cluster, eigenvectors are in rotation -- PROBLEM - for some reason had to convert everything to numeric  
    pca_NELF_1 <- NELF_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_NELF_2 <- NELF_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_NELF_3 <- NELF_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>%   
      prcomp(center = TRUE, scale = TRUE)

    pca_NLF_1 <- NLF_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_NLF_2 <- NLF_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_NLF_3 <- NLF_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)

    pca_Sputum_1 <- Sputum_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_Sputum_2 <- Sputum_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_Sputum_3 <- Sputum_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)

    #eigenvector dfs of first principal component 
    eigencytokines_NELF_1 <- data.frame(pca_NELF_1$rotation[,"PC1"])
      colnames(eigencytokines_NELF_1)[1] <- "ClusterA"
    eigencytokines_NELF_2 <- data.frame(pca_NELF_2$rotation[,"PC1"])
      colnames(eigencytokines_NELF_2)[1] <- "ClusterB"
    eigencytokines_NELF_3 <- data.frame(pca_NELF_3$rotation[,"PC1"])
      colnames(eigencytokines_NELF_3)[1] <- "ClusterC"

    eigencytokines_NLF_1 <- data.frame(pca_NLF_1$rotation[,"PC1"])
      colnames(eigencytokines_NLF_1)[1] <- "ClusterA"
    eigencytokines_NLF_2 <- data.frame(pca_NLF_2$rotation[,"PC1"])
      colnames(eigencytokines_NLF_2)[1] <- "ClusterB"
    eigencytokines_NLF_3 <- data.frame(pca_NLF_3$rotation[,"PC1"])
      colnames(eigencytokines_NLF_3)[1] <- "ClusterC"

    eigencytokines_Sputum_1 <- data.frame(pca_Sputum_1$rotation[,"PC1"])
      colnames(eigencytokines_Sputum_1)[1] <- "ClusterA"
    eigencytokines_Sputum_2 <- data.frame(pca_Sputum_2$rotation[,"PC1"])
      colnames(eigencytokines_Sputum_2)[1] <- "ClusterB"
    eigencytokines_Sputum_3 <- data.frame(pca_Sputum_3$rotation[,"PC1"])
      colnames(eigencytokines_Sputum_3)[1] <- "ClusterC"

    #collapse all eigencytokine dfs
    eigencytokines_NELF <- cbind(eigencytokines_NELF_1, eigencytokines_NELF_2, eigencytokines_NELF_3)
    eigencytokines_NLF <- cbind(eigencytokines_NLF_1, eigencytokines_NLF_2, eigencytokines_NLF_3)
    eigencytokines_Sputum <- cbind(eigencytokines_Sputum_1, eigencytokines_Sputum_2, eigencytokines_Sputum_3)
    
    all_eigencytokines = cbind(eigencytokines_NLF, eigencytokines_NELF, eigencytokines_Sputum)
    return(all_eigencytokines)
}

#calling fn
eigencytokines_NLF_NS = get_eigencytokines("NS")[1:3]
eigencytokines_NELF_NS = get_eigencytokines("NS")[4:6]
eigencytokines_Sputum_NS = get_eigencytokines("NS")[7:9]
eigencytokines_NLF_Ecig = get_eigencytokines("Ecig")[1:3]
eigencytokines_NELF_Ecig = get_eigencytokines("Ecig")[4:6]
eigencytokines_Sputum_Ecig = get_eigencytokines("Ecig")[7:9]
eigencytokines_NLF_CS = get_eigencytokines("CS")[1:3]
eigencytokines_NELF_CS = get_eigencytokines("CS")[4:6]
eigencytokines_Sputum_CS = get_eigencytokines("CS")[7:9]

In [3]:
#exporting eigencytokines these are different than the eigencytokines from the baseline analysis, because these
#now include CS and Ecig users
eigencytokines_NLF = rbind(eigencytokines_NLF_NS, eigencytokines_NLF_CS, eigencytokines_NLF_Ecig)
eigencytokines_NELF = rbind(eigencytokines_NELF_NS, eigencytokines_NELF_CS, eigencytokines_NELF_Ecig)
eigencytokines_Sputum = rbind(eigencytokines_Sputum_NS, eigencytokines_Sputum_CS, eigencytokines_Sputum_Ecig)

# write.csv(eigencytokines_NLF, paste0(Output,"/", cur_date, "_NLF_eigencytokines.csv"), row.names = TRUE)
# write.csv(eigencytokines_NELF, paste0(Output,"/", cur_date, "_NELF_eigencytokines.csv"), row.names = TRUE)
# write.csv(eigencytokines_Sputum, paste0(Output,"/", cur_date, "_Sputum_eigencytokines.csv"), row.names = TRUE)

# Wilcoxon Rank Sum tests

Comparing eigencytokines of non-smokers to smokers. 

In [4]:
#converting subject ids to col, melting, and adding compartment
changed_df = function(df, compartment_name){
    df = reshape2::melt(df %>%
        rownames_to_column(var = "SubjectID"), variable = "Cluster",  value.name = 'Conc_pslog2')
    df$Compartment = rep(compartment_name, times = length(df$SubjectID))
    return(df)
}
NS_eigencytokines_NLF = changed_df(eigencytokines_NLF_NS,"NLF")
NS_eigencytokines_NELF = changed_df(eigencytokines_NELF_NS,"NELF")
NS_eigencytokines_Sputum = changed_df(eigencytokines_Sputum_NS,"Sputum")
Ecig_eigencytokines_NLF = changed_df(eigencytokines_NLF_Ecig,"NLF")
Ecig_eigencytokines_NELF = changed_df(eigencytokines_NELF_Ecig,"NELF")
Ecig_eigencytokines_Sputum = changed_df(eigencytokines_Sputum_Ecig,"Sputum")
CS_eigencytokines_NLF = changed_df(eigencytokines_NLF_CS,"NLF")
CS_eigencytokines_NELF = changed_df(eigencytokines_NELF_CS,"NELF")
CS_eigencytokines_Sputum = changed_df(eigencytokines_Sputum_CS,"Sputum")

#recombining into 3 dataframes
NS_eigencytokines = rbind(NS_eigencytokines_NLF, NS_eigencytokines_NELF, NS_eigencytokines_Sputum)
CS_eigencytokines = rbind(CS_eigencytokines_NLF, CS_eigencytokines_NELF, CS_eigencytokines_Sputum)
Ecig_eigencytokines = rbind(Ecig_eigencytokines_NLF, Ecig_eigencytokines_NELF, Ecig_eigencytokines_Sputum)

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables



In [5]:
#creating vectors to loop through
compartment = c('NLF','NELF','Sputum')
cluster = unique(NS_eigencytokines_NLF$Cluster)

#initializing vectors to store values
CS_df = data.frame()
Ecig_df = data.frame()
CS_Ecig_df = data.frame()
wilcoxon_rank_sum_values = function(df1, df2, empty_df){
    #running wilcoxon rank sum and storing the statistic, compartment, cluster, and p value in a vector
    for (i in 1:length(compartment)){
        for (j in 1: length(cluster)){
            variable1_df = df1 %>% # baseline df
                filter(Compartment == compartment[i], Cluster == cluster[j]) %>%
                select(Compartment, Cluster, Conc_pslog2)
            variable2_df = df2 %>% # smoker df
                filter(Compartment == compartment[i], Cluster == cluster[j]) %>%
                select(Compartment, Cluster, Conc_pslog2)


            #running wilcoxon rank sum
            wilcox_test = wilcox.test(variable1_df$Conc_pslog2, variable2_df$Conc_pslog2)
            
            #calculating absolute difference
            AD = (mean(variable2_df$Conc_pslog2) - mean(variable1_df$Conc_pslog2))

            #contains compartment, cluster, u stat, p value
            values_df = cbind(compartment[i], as.character(cluster[j]), AD, wilcox_test$statistic, wilcox_test$p.value)
            empty_df = rbind(empty_df, values_df)

        }
    }
    colnames(empty_df) = c("Compartment",'Cluster', 'AD','Stat', 'P Value')
    return(empty_df)
}

#calling fn
CS_wilcoxon_values = wilcoxon_rank_sum_values(NS_eigencytokines, CS_eigencytokines, CS_df)
Ecig_wilcoxon_values = wilcoxon_rank_sum_values(NS_eigencytokines, Ecig_eigencytokines, Ecig_df)
CS_Ecig_wilcoxon_values = wilcoxon_rank_sum_values(CS_eigencytokines, Ecig_eigencytokines, CS_Ecig_df)

In [6]:
final_table = function(df){
    #"""
    #Adding a padj col
    
    #:param: vector, demographic variable
    #:output: a 6x12 df containing compartment, cluster, protein, u stat, p value, p adj CHANGE THIS!!

    #"""
    
    PAdj = c()
    for (i in 1:length(compartment)){
        single_compartment_df = df %>%
            filter(Compartment == compartment[i])
        padj =  p.adjust(as.numeric(as.character(single_compartment_df$`P Value`)), method = "fdr") 
        PAdj = c(PAdj, padj)
        
    }
    
    df = cbind(df, PAdj)
    return(df)
}

#calling fn
CS_table = final_table(CS_wilcoxon_values)
CS_table
Ecig_table = final_table(Ecig_wilcoxon_values)
Ecig_table
CS_Ecig_table = final_table(CS_Ecig_wilcoxon_values)
CS_Ecig_table

,Compartment,Cluster,AD,Stat,P Value,PAdj
,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>
W,NLF,ClusterA,-0.0145323953683299,97,0.792955235488551,0.7929552
W1,NLF,ClusterB,-0.145857578442958,116,0.238822532318292,0.3582338
W2,NLF,ClusterC,-0.123670513346295,119,0.185184786347796,0.3582338
W3,NELF,ClusterA,-0.00388516073406031,103,0.58264369363306,0.9429524
W4,NELF,ClusterB,-0.026370912963717,93,0.942952393772154,0.9429524
W5,NELF,ClusterC,-0.0106803302321949,98,0.756376761739529,0.9429524
W6,Sputum,ClusterA,0.0391139960871419,89,0.942952393772154,0.9429524
W7,Sputum,ClusterB,-0.10051572435611,111,0.34988438701186,0.5248266
W8,Sputum,ClusterC,-0.0148431078782217,114,0.279954033990917,0.5248266


,Compartment,Cluster,AD,Stat,P Value,PAdj
,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>
W,NLF,ClusterA,0.479332070360599,0,7.54197509809517e-09,2.262593e-08
W1,NLF,ClusterB,-0.162046186408269,143,0.356113292156035,5.341699e-01
W2,NLF,ClusterC,-0.122510267739814,133,0.597070760978688,5.970708e-01
W3,NELF,ClusterA,0.0200142346341384,83,0.160975124586358,3.771206e-01
W4,NELF,ClusterB,0.0313065451559485,96,0.377120619090568,3.771206e-01
W5,NELF,ClusterC,0.0128519141367842,90,0.262117317119595,3.771206e-01
W6,Sputum,ClusterA,0.0246558747804312,126,0.799302563394779,7.993026e-01
W7,Sputum,ClusterB,0.189556748126578,58,0.0147413936872349,4.422418e-02
W8,Sputum,ClusterC,0.0193029611027928,84,0.173340434102888,2.600107e-01


,Compartment,Cluster,AD,Stat,P Value,PAdj
,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>
W,NLF,ClusterA,0.493864465728929,0,1.67000877172107e-08,5.010026e-08
W1,NLF,ClusterB,-0.0161886079653113,105,0.836755406757774,8.367554e-01
W2,NLF,ClusterC,0.00116024560648087,93,0.482564398669504,7.238466e-01
W3,NELF,ClusterA,0.0238993953681987,72,0.11266811038925,2.628632e-01
W4,NELF,ClusterB,0.0576774581196655,83,0.262863154888721,2.628632e-01
W5,NELF,ClusterC,0.0235322443689791,80,0.212763826942001,2.628632e-01
W6,Sputum,ClusterA,-0.0144581213067106,125,0.563403411076417,5.634034e-01
W7,Sputum,ClusterB,0.290072472482688,35,0.00103954706022093,3.118641e-03
W8,Sputum,ClusterC,0.0341460689810145,61,0.0389420661432024,5.841310e-02


In [7]:
#exporting tables
write.csv(CS_table, paste0(Output,"/", cur_date, "_CS_Distribution_Analysis.csv"), row.names = FALSE)
write.csv(Ecig_table, paste0(Output,"/", cur_date, "_Ecig_Distribution_Analysis.csv"), row.names = FALSE)
write.csv(CS_Ecig_table, paste0(Output,"/", cur_date, "_CS_Ecig_Distribution_Analysis.csv"), row.names = FALSE)